In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_timestamp, to_date, split, when, col, sum, mean, round, count, abs, regexp_extract, date_format
import numpy as np
from datetime import datetime, timedelta
from pyspark.sql.window import Window
from pyspark.sql import functions as F

In [2]:
spark = SparkSession.builder \
    .appName("machungkhoan") \
    .config("spark.cores.max", "2") \
    .config("spark.executor.memory", "4g") \
    .getOrCreate()

24/05/24 03:01:53 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [3]:
# Đọc dữ liệu từ bảng nguồn machungkhoan
df_evalution = spark.read.format("iceberg").load("stock_db.datn_danhgiachungkhoan")

# Đọc dữ liệu từ bảng nguồn machungkhoan
df_machungkhoan = spark.read.format("iceberg").load("stock_db.datn_machungkhoan")

In [4]:
df_evalution.show(1)

+---+---+----------+----------+-------+-------+----------+----------+------+-------------------+
| pe| pb|industrype|industrypb|indexpe|indexpb| from_date|   to_date|symbol|         ky_du_lieu|
+---+---+----------+----------+-------+-------+----------+----------+------+-------------------+
|0.0|0.0|      14.4|       2.0|   16.2|    2.1|2019-05-15|2019-05-15|   VPW|15/05/2024 01:05:43|
+---+---+----------+----------+-------+-------+----------+----------+------+-------------------+
only showing top 1 row



In [29]:
dim_stock_df  = df_machungkhoan.filter((col('categoryname').isNotNull()) & (col('categoryname') != ""))
df_stock_drop_duplicate = dim_stock_df.dropDuplicates(['symbol'])
# Tiếp theo, ghi dữ liệu vào bảng dim_stock
dim_stock_data = df_stock_drop_duplicate.select(
    col("symbol").alias("stocksymbol"),
    "companyname",
    "categoryid"
).distinct()

# Convert dim_stock_df to a list of stocksymbols
valid_stocksymbols = [row.stocksymbol for row in dim_stock_data.select("stocksymbol").distinct().collect()]

In [35]:
df_evalution = df_evalution.withColumn("from_date", to_date(col("from_date"), "yyyy-MM-dd"))
df_evalution = df_evalution.withColumn("to_date", to_date(col("to_date"), "yyyy-MM-dd"))


In [36]:

df_evalution = df_evalution.withColumn("fromdateid", date_format(col("from_date"), "ddMMyyyy").cast("int"))

df_evalution = df_evalution.withColumn("todateid", date_format(col("to_date"), "ddMMyyyy").cast("int"))

In [37]:
df_evalution.show(1)

+---+---+----------+----------+-------+-------+----------+----------+------+-------------------+----------+--------+
| pe| pb|industrype|industrypb|indexpe|indexpb| from_date|   to_date|symbol|         ky_du_lieu|fromdateid|todateid|
+---+---+----------+----------+-------+-------+----------+----------+------+-------------------+----------+--------+
|0.0|0.0|      14.8|       2.2|   16.2|    2.1|2019-05-15|2019-05-15|   VTK|15/05/2024 01:08:07|  15052019|15052019|
+---+---+----------+----------+-------+-------+----------+----------+------+-------------------+----------+--------+
only showing top 1 row



In [38]:
# Select and rename columns to match the schema of 'fact_price_history'
df_evalution_selected = df_evalution.select(
    col("symbol").alias("stocksymbol"),
    col("fromdateid"),
    col("todateid"),
    col("pe").cast("decimal(18, 2)"),
    col("pb").cast("decimal(18, 2)"),
    col("industrype").cast("decimal(18, 2)"),
    col("industrypb").cast("decimal(18, 2)"),
    col("indexpe").cast("decimal(18, 2)"),
    col("indexpb").cast("decimal(18, 2)")
)

In [39]:
df_evalution_selected.show(1)

+-----------+----------+--------+------+----+----------+----------+-------+-------+
|stocksymbol|fromdateid|todateid|    pe|  pb|industrype|industrypb|indexpe|indexpb|
+-----------+----------+--------+------+----+----------+----------+-------+-------+
|        VTC|  15052019|15052019|-10.60|0.40|     10.70|      1.40|  16.20|   2.10|
+-----------+----------+--------+------+----+----------+----------+-------+-------+
only showing top 1 row



In [40]:
df_evalution_filtered = df_evalution_selected.filter(col("stocksymbol").isin(valid_stocksymbols))

In [ ]:
# Ghi kết quả vào bảng `fact_stock_indicator`
df_evalution_filtered.write \
    .format("jdbc") \
    .option("driver", "com.mysql.cj.jdbc.Driver") \
    .option("url", "jdbc:mysql://10.168.6.106:3306/dtm_stock") \
    .option("dbtable", "fact_evaluation") \
    .option("user", "acc_etl") \
    .option("password", "Vnpt123456") \
    .mode("append") \
    .save()